In [18]:
import numpy as np
import scipy as sp

import tensorflow as tf
from tensorflow import keras

import tf2onnx
import onnx
import os

## Choose computation device (CPU)

In [2]:
physical_devices = tf.config.list_physical_devices()
print(f"These are the physical devices available:\n{physical_devices}")

try:
    # Disable all GPUS
    tf.config.set_visible_devices([], 'GPU')
    visible_devices = tf.config.get_visible_devices()
    print(f"These are the visible devices:\n{visible_devices}")
except:
    pass

These are the physical devices available:
[PhysicalDevice(name='/physical_device:CPU:0', device_type='CPU')]
These are the visible devices:
[PhysicalDevice(name='/physical_device:CPU:0', device_type='CPU')]


## User inputs

In [3]:
# EDIT THIS SECTION FOR USER INPUTS
#
name = 'model_0'
in_file = '../data/ts9_test1_in_FP32.wav'
out_file = '../data/ts9_test1_out_FP32.wav'
epochs = 1

train_mode = 0     # 0 = speed training, 
                   # 1 = accuracy training 
                   # 2 = extended training

input_size = 150 
batch_size = 4096 
test_size = 0.2

if not os.path.exists('models/'+name):
    os.makedirs('models/'+name)
else:
    print("A model with the same name already exists. Please choose a new name.")
    exit

A model with the same name already exists. Please choose a new name.


## Define some helper functions

In [4]:
def save_wav(name, data):
    sp.io.wavfile.write(name, 44100, data.flatten().astype(np.float32))

def normalize(data):
    data_max = max(data)
    data_min = min(data)
    data_norm = max(data_max,abs(data_min))
    return data / data_norm

## Pre-processing the data

In [5]:
# Load and Preprocess Data ###########################################
in_rate, in_data = sp.io.wavfile.read(in_file)
out_rate, out_data = sp.io.wavfile.read(out_file)

X_all = in_data.astype(np.float32).flatten()  
X_all = normalize(X_all).reshape(len(X_all),1)   
y_all = out_data.astype(np.float32).flatten() 
y_all = normalize(y_all).reshape(len(y_all),1)

# Get the last 20% of the wav data for testing and thee rest for training
X_training, X_testing = np.split(X_all, [int(len(X_all)*(1-test_size))])
y_training, y_testing = np.split(y_all, [int(len(y_all)*(1-test_size))])
print(f"X_training shape (pre-processing): {X_training.shape}")
print(f"y_training shape (pre-processing): {y_training.shape}")
print(f"X_testing shape (pre-processing): {X_testing.shape}")
print(f"y_testing shape (pre-processing): {y_testing.shape}")

# Create a new array where each element is an array of input_size samples in time order
# Each element of the new array is shifted by one sample from the previous element
indices = np.arange(input_size) + np.arange(len(X_training)-input_size+1)[:,np.newaxis]

# We need to split the data otherwise the tf.gather function will throw an error since we run out of memory
X_ordered_training = tf.gather(X_training,indices[:len(indices)//2])
X_ordered_training = tf.concat([X_ordered_training, tf.gather(X_training,indices[len(indices)//2:])], axis=0)
print(f"X_ordered_training shape: {X_ordered_training.shape}")

indices = np.arange(input_size) + np.arange(len(X_testing)-input_size+1)[:,np.newaxis]
X_ordered_testing = tf.gather(X_testing,indices) 
print(f"X_ordered_testing shape: {X_ordered_testing.shape}")

# The input size defines the number of samples used for each prediction
# Therefore the first output value that we get is at index input_size-1
y_ordered_training = y_training[input_size-1:]
print(f"y_ordered_training shape: {y_ordered_training.shape}")
y_ordered_testing = y_testing[input_size-1:]
print(f"y_ordered_testing shape: {y_ordered_testing.shape}")


shuffled_indices = np.random.permutation(len(X_ordered_training)) 
X_random_training = tf.gather(X_ordered_training, shuffled_indices)
y_random_training = tf.gather(y_ordered_training, shuffled_indices)
print(f"X_random_training shape (post-processing): {X_random_training.shape}")
print(f"y_random_training shape (post-processing): {y_random_training.shape}")

print(f"The X_random_training data is an array, where each element is an array of input_size samples in time order. Therefore the lenght is smaller than the original X_training array (the first {input_size} samples are grouped).")
print(f"The y_random_training data is an array, where each element is a single sample. This single sample is the target output for the corresponding X_random_training element, which consists of input samples.")

X_training shape (pre-processing): (6587907, 1)
y_training shape (pre-processing): (6587907, 1)
X_testing shape (pre-processing): (1646977, 1)
y_testing shape (pre-processing): (1646977, 1)


2024-03-08 13:58:20.534818: W external/local_tsl/tsl/framework/cpu_allocator_impl.cc:83] Allocation of 3952654800 exceeds 10% of free system memory.
2024-03-08 13:58:21.155171: W external/local_tsl/tsl/framework/cpu_allocator_impl.cc:83] Allocation of 1976327400 exceeds 10% of free system memory.
2024-03-08 13:58:21.625643: W external/local_tsl/tsl/framework/cpu_allocator_impl.cc:83] Allocation of 3952654800 exceeds 10% of free system memory.
2024-03-08 13:58:22.294189: W external/local_tsl/tsl/framework/cpu_allocator_impl.cc:83] Allocation of 1976327400 exceeds 10% of free system memory.
2024-03-08 13:58:22.753254: W external/local_tsl/tsl/framework/cpu_allocator_impl.cc:83] Allocation of 3952654800 exceeds 10% of free system memory.


X_ordered_training shape: (6587758, 150, 1)
X_ordered_testing shape: (1646828, 150, 1)
y_ordered_training shape: (6587758, 1)
y_ordered_testing shape: (1646828, 1)
X_random_training shape (post-processing): (6587758, 150, 1)
y_random_training shape (post-processing): (6587758, 1)
The X_random_training data is an array, where each element is an array of input_size samples in time order. Therefore the lenght is smaller than the original X_training array (the first 150 samples are grouped).
The y_random_training data is an array, where each element is a single sample. This single sample is the target output for the corresponding X_random_training element, which consists of input samples.


## Define the model

In [6]:
'''This is a similar Tensorflow/Keras implementation of the LSTM model from the paper:
    "Real-Time Guitar Amplifier Emulation with Deep Learning"
    https://www.mdpi.com/2076-3417/10/3/766/htm

    Uses a stack of two 1-D Convolutional layers, followed by LSTM, followed by 
    a Dense (fully connected) layer. Three preset training modes are available, 
    with further customization by editing the code. A Functional keras model 
    is implemented here.

    Note: RAM may be a limiting factor for the parameter "input_size". The wav data
      is preprocessed and stored in RAM, which improves training speed but quickly runs out
      if using a large number for "input_size".  Reduce this if you are experiencing
      RAM issues.
'''

if train_mode == 0:         # Speed Training
    learning_rate = 0.01 
    conv1d_strides = 12   
    conv1d_1_strides = 12
    conv1d_filters = 16
    hidden_units = 36
elif train_mode == 1:       # Accuracy Training (~10x longer than Speed Training)
    learning_rate = 0.01 
    conv1d_strides = 4
    conv1d_filters = 36
    hidden_units= 64
else:                       # Extended Training (~60x longer than Accuracy Training)
    learning_rate = 0.0005 
    conv1d_strides = 3
    conv1d_filters = 36
    hidden_units= 96

# Create Functional Model ###########################################
keras.backend.clear_session()

class NeuralNetwork(tf.keras.Model):
    def __init__(self, input_size, conv1d_filters, conv1d_strides, hidden_units, learning_rate):
        super(NeuralNetwork, self).__init__()
        self.conv1d_1 = keras.layers.Conv1D(filters=conv1d_filters, kernel_size=12, strides=conv1d_strides, activation=None, batch_size=1)
        self.conv1d_2 = keras.layers.Conv1D(filters=conv1d_filters, kernel_size=12, strides=conv1d_strides, activation=None, batch_size=1)
        self.lstm = keras.layers.LSTM(units=hidden_units, return_sequences=False, stateful=False, batch_size=batch_size, kernel_initializer='glorot_uniform', recurrent_initializer='zeros', bias_initializer='zeros', use_bias=True)
        self.dense = keras.layers.Dense(units=1, activation=None, batch_size=1)

        self.build((1, input_size, 1))

    def call(self, inputs, training=False):
        x = keras.layers.ZeroPadding1D(padding=12, batch_size=1)(inputs)
        x = self.conv1d_1(x)
        x = keras.layers.ZeroPadding1D(padding=12, batch_size=1)(x)
        x = self.conv1d_2(x)
        x = self.lstm(x)
        return self.dense(x)

# Define the input shape
inputs = keras.Input(shape=(input_size,1), batch_size=1)

model = NeuralNetwork(input_size, conv1d_filters, conv1d_strides, hidden_units, learning_rate)


model.compile(optimizer=keras.optimizers.Adam(learning_rate=learning_rate), loss='mse')
model.summary()

Model: "neural_network"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 conv1d (Conv1D)             multiple                  208       
                                                                 
 conv1d_1 (Conv1D)           multiple                  3088      
                                                                 
 lstm (LSTM)                 multiple                  7632      
                                                                 
 dense (Dense)               multiple                  37        
                                                                 
Total params: 10965 (42.83 KB)
Trainable params: 10965 (42.83 KB)
Non-trainable params: 0 (0.00 Byte)
_________________________________________________________________


## Train the model

In [7]:
history = model.fit(x=X_random_training, y=y_random_training, epochs=epochs, batch_size=batch_size, validation_data=(X_ordered_testing, y_ordered_testing)) 

1609/1609 [==============================] - 43s 25ms/step - loss: 8.0674e-04 - val_loss: 3.0798e-04


In [8]:
model.save_weights('models/'+name+'/'+name)

## Run predictions
### 0. Load the model

In [9]:
model.load_weights('models/'+name+'/'+name)

### 1. On the test audio data

In [10]:
# Run Prediction #################################################
# Test the model on the testing data #############################

print("Running prediction..")
prediction = model.predict(X_ordered_testing)

save_wav('models/'+name+'/y_pred.wav', prediction)
save_wav('models/'+name+'/x_test.wav', X_testing)
save_wav('models/'+name+'/y_test.wav', y_testing)

print("X_testing shape: ", X_testing.shape)
print("X_ordered_testing shape: ", X_ordered_testing.shape)
print("y_testing shape: ", y_testing.shape)
print("prediction shape: ", prediction.shape)

print("Note that the prediction shape is smaller than the y_testing shape. This is because the first predicted sample needs input_size samples for prediction.\n")

Running prediction..
51464/51464 [==============================] - 67s 1ms/step
X_testing shape:  (1646977, 1)
X_ordered_testing shape:  (1646828, 150, 1)
y_testing shape:  (1646977, 1)
prediction shape:  (1646828, 1)
Note that the prediction shape is smaller than the y_testing shape. This is because the first predicted sample needs input_size samples for prediction.



### 2. On a number sequence (to control inference)

In [11]:
# Test the model simple number sequence to compare with inference #
X_testing_2 = np.array([])

for i in range(0, 300):
    X_testing_2 = np.append(X_testing_2, i*0.001)

X_testing_2 = np.expand_dims(X_testing_2, axis=0)
X_testing_2 = np.expand_dims(X_testing_2, axis=0)

X_testing_2 = np.reshape(X_testing_2, (2, 150, 1))

print("Running prediction..")
prediction_2 = model.predict(X_testing_2)
print(f"prediction {prediction_2}")

print("X_testing_2 shape: ", X_testing_2.shape)
print("prediction_2 shape: ", prediction_2.shape)

Running prediction..
1/1 [==============================] - 0s 13ms/step
prediction [[-0.15528308]
 [-0.2584993 ]]
X_testing_2 shape:  (2, 150, 1)
prediction_2 shape:  (2, 1)


## Export as tflite model

In [22]:
# tflite does not support dynamic batch sizes, so we need to create a model for each batch size

buffer_sizes = [64, 128, 256, 512, 1024, 2048, 4096, 8192]
model_input_size = 150
# rf is the receptive field and hence the kernel size of the last layer because of the dilation
# hence the model input size needs to be at least rf
for buffer_size in buffer_sizes:
    batch_size = buffer_size
    input_shape = [batch_size, model_input_size, 1]

    func = tf.function(model).get_concrete_function(
        tf.TensorSpec(input_shape, dtype=tf.float32))
    converter = tf.lite.TFLiteConverter.from_concrete_functions([func], model)
    tflite_model = converter.convert()

    # Save the model.
    with open("models/"+name+"/"+"GuitarLSTM-"+str(buffer_size)+".tflite", 'wb') as f:
      f.write(tflite_model)

INFO:tensorflow:Assets written to: /tmp/tmpf_ywq6r3/assets


INFO:tensorflow:Assets written to: /tmp/tmpf_ywq6r3/assets
2024-03-08 14:16:14.984963: W tensorflow/compiler/mlir/lite/python/tf_tfl_flatbuffer_helpers.cc:378] Ignored output_format.
2024-03-08 14:16:14.984983: W tensorflow/compiler/mlir/lite/python/tf_tfl_flatbuffer_helpers.cc:381] Ignored drop_control_dependency.
2024-03-08 14:16:14.985126: I tensorflow/cc/saved_model/reader.cc:83] Reading SavedModel from: /tmp/tmpf_ywq6r3
2024-03-08 14:16:14.990758: I tensorflow/cc/saved_model/reader.cc:51] Reading meta graph with tags { serve }
2024-03-08 14:16:14.990768: I tensorflow/cc/saved_model/reader.cc:146] Reading SavedModel debug info (if present) from: /tmp/tmpf_ywq6r3
2024-03-08 14:16:15.006423: I tensorflow/cc/saved_model/loader.cc:233] Restoring SavedModel bundle.
2024-03-08 14:16:15.070747: I tensorflow/cc/saved_model/loader.cc:217] Running initialization op on SavedModel bundle at path: /tmp/tmpf_ywq6r3
2024-03-08 14:16:15.109141: I tensorflow/cc/saved_model/loader.cc:316] SavedModel

INFO:tensorflow:Assets written to: /tmp/tmpyy4kmsej/assets


INFO:tensorflow:Assets written to: /tmp/tmpyy4kmsej/assets
2024-03-08 14:16:27.492635: W tensorflow/compiler/mlir/lite/python/tf_tfl_flatbuffer_helpers.cc:378] Ignored output_format.
2024-03-08 14:16:27.492660: W tensorflow/compiler/mlir/lite/python/tf_tfl_flatbuffer_helpers.cc:381] Ignored drop_control_dependency.
2024-03-08 14:16:27.492839: I tensorflow/cc/saved_model/reader.cc:83] Reading SavedModel from: /tmp/tmpyy4kmsej
2024-03-08 14:16:27.498338: I tensorflow/cc/saved_model/reader.cc:51] Reading meta graph with tags { serve }
2024-03-08 14:16:27.498351: I tensorflow/cc/saved_model/reader.cc:146] Reading SavedModel debug info (if present) from: /tmp/tmpyy4kmsej
2024-03-08 14:16:27.515862: I tensorflow/cc/saved_model/loader.cc:233] Restoring SavedModel bundle.
2024-03-08 14:16:27.582341: I tensorflow/cc/saved_model/loader.cc:217] Running initialization op on SavedModel bundle at path: /tmp/tmpyy4kmsej
2024-03-08 14:16:27.623338: I tensorflow/cc/saved_model/loader.cc:316] SavedModel

INFO:tensorflow:Assets written to: /tmp/tmp7cgp5vgp/assets


...

INFO:tensorflow:Assets written to: /tmp/tmp6ns0_a_v/assets
2024-03-08 14:17:18.221699: W tensorflow/compiler/mlir/lite/python/tf_tfl_flatbuffer_helpers.cc:378] Ignored output_format.
2024-03-08 14:17:18.221719: W tensorflow/compiler/mlir/lite/python/tf_tfl_flatbuffer_helpers.cc:381] Ignored drop_control_dependency.
2024-03-08 14:17:18.221860: I tensorflow/cc/saved_model/reader.cc:83] Reading SavedModel from: /tmp/tmp6ns0_a_v
2024-03-08 14:17:18.227178: I tensorflow/cc/saved_model/reader.cc:51] Reading meta graph with tags { serve }
2024-03-08 14:17:18.227188: I tensorflow/cc/saved_model/reader.cc:146] Reading SavedModel debug info (if present) from: /tmp/tmp6ns0_a_v
2024-03-08 14:17:18.242415: I tensorflow/cc/saved_model/loader.cc:233] Restoring SavedModel bundle.
2024-03-08 14:17:18.306527: I tensorflow/cc/saved_model/loader.cc:217] Running initialization op on SavedModel bundle at path: /tmp/tmp6ns0_a_v
2024-03-08 14:17:18.344425: I tensorflow/cc/saved_model/loader.cc:316] SavedModel

INFO:tensorflow:Assets written to: /tmp/tmpsz7qm609/assets


INFO:tensorflow:Assets written to: /tmp/tmpsz7qm609/assets
2024-03-08 14:17:30.562638: W tensorflow/compiler/mlir/lite/python/tf_tfl_flatbuffer_helpers.cc:378] Ignored output_format.
2024-03-08 14:17:30.562657: W tensorflow/compiler/mlir/lite/python/tf_tfl_flatbuffer_helpers.cc:381] Ignored drop_control_dependency.
2024-03-08 14:17:30.562804: I tensorflow/cc/saved_model/reader.cc:83] Reading SavedModel from: /tmp/tmpsz7qm609
2024-03-08 14:17:30.568282: I tensorflow/cc/saved_model/reader.cc:51] Reading meta graph with tags { serve }
2024-03-08 14:17:30.568293: I tensorflow/cc/saved_model/reader.cc:146] Reading SavedModel debug info (if present) from: /tmp/tmpsz7qm609
2024-03-08 14:17:30.584497: I tensorflow/cc/saved_model/loader.cc:233] Restoring SavedModel bundle.
2024-03-08 14:17:30.648224: I tensorflow/cc/saved_model/loader.cc:217] Running initialization op on SavedModel bundle at path: /tmp/tmpsz7qm609
2024-03-08 14:17:30.686734: I tensorflow/cc/saved_model/loader.cc:316] SavedModel

INFO:tensorflow:Assets written to: /tmp/tmpkoxhytga/assets


INFO:tensorflow:Assets written to: /tmp/tmpkoxhytga/assets
2024-03-08 14:17:43.311805: W tensorflow/compiler/mlir/lite/python/tf_tfl_flatbuffer_helpers.cc:378] Ignored output_format.
2024-03-08 14:17:43.311826: W tensorflow/compiler/mlir/lite/python/tf_tfl_flatbuffer_helpers.cc:381] Ignored drop_control_dependency.
2024-03-08 14:17:43.311968: I tensorflow/cc/saved_model/reader.cc:83] Reading SavedModel from: /tmp/tmpkoxhytga
2024-03-08 14:17:43.317513: I tensorflow/cc/saved_model/reader.cc:51] Reading meta graph with tags { serve }
2024-03-08 14:17:43.317525: I tensorflow/cc/saved_model/reader.cc:146] Reading SavedModel debug info (if present) from: /tmp/tmpkoxhytga
2024-03-08 14:17:43.333436: I tensorflow/cc/saved_model/loader.cc:233] Restoring SavedModel bundle.
2024-03-08 14:17:43.397171: I tensorflow/cc/saved_model/loader.cc:217] Running initialization op on SavedModel bundle at path: /tmp/tmpkoxhytga
2024-03-08 14:17:43.435825: I tensorflow/cc/saved_model/loader.cc:316] SavedModel

In [23]:
for buffer_size in buffer_sizes:
    batch_size = buffer_size
    input_shape = [batch_size, model_input_size, 1]

    # Define the input shape
    input_signature = [tf.TensorSpec(input_shape, tf.float32, name='x')]

    # Convert the model
    onnx_model, _ = tf2onnx.convert.from_keras(model, input_signature, opset=18)
    onnx.save(proto=onnx_model, f="models/"+name+"/"+"GuitarLSTM-tflite-"+str(buffer_size)+".onnx")

2024-03-08 14:18:22.800893: I tensorflow/core/grappler/devices.cc:66] Number of eligible GPUs (core count >= 8, compute capability >= 0.0): 0
2024-03-08 14:18:22.800971: I tensorflow/core/grappler/clusters/single_machine.cc:361] Starting new session
2024-03-08 14:18:23.034716: I tensorflow/core/grappler/devices.cc:66] Number of eligible GPUs (core count >= 8, compute capability >= 0.0): 0
2024-03-08 14:18:23.034799: I tensorflow/core/grappler/clusters/single_machine.cc:361] Starting new session
2024-03-08 14:18:23.695070: I tensorflow/core/grappler/devices.cc:66] Number of eligible GPUs (core count >= 8, compute capability >= 0.0): 0
2024-03-08 14:18:23.695138: I tensorflow/core/grappler/clusters/single_machine.cc:361] Starting new session
2024-03-08 14:18:24.328572: I tensorflow/core/grappler/devices.cc:66] Number of eligible GPUs (core count >= 8, compute capability >= 0.0): 0
2024-03-08 14:18:24.328685: I tensorflow/core/grappler/clusters/single_machine.cc:361] Starting new session
